In [150]:
# %%bash

# pip install --upgrade pip
# pip install farm-haystack[colab,inference]


In [ ]:



from haystack.telemetry import tutorial_running

tutorial_running(1)


In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


In [ ]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:
# from haystack.utils import fetch_archive_from_http

from google.colab import files

uploaded = files.upload()



Saving testingtexts.zip to testingtexts.zip


In [ ]:
import shutil
import os

zip_file_name = 'testingtexts.zip'
target_dir = '/content/data/text'
shutil.unpack_archive(zip_file_name, target_dir, 'zip')


In [ ]:
# Optional step
import shutil

directory_path = '/content/data/text/__MACOSX'
shutil.rmtree(directory_path)


In [ ]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [target_dir+ "/" + f for f in os.listdir(target_dir)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)


INFO:haystack.pipelines.base:It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.
Updating BM25 representation...: 100%|██████████| 285/285 [00:00<00:00, 4425.29 docs/s]


{'documents': [<Document: {'content': "INTRODUCTION\nThe Vajont reservoir disaster is a classic example of the consequences of the failure of engineers and geologists to understand the nature of the problem that they were trying to deal with. During the filling of the reservoir a block of approximately 270 million m3\ndetached from one wall and slid into the lake at velocities of up to 30 m sec-1 (approx. 110 km h-1). As a result a wave over topped the dam by 250 m and swept onto the valley below, with the loss of about 2500 lives. Remarkably the dam remained unbroken by the flood.\nLOCATION AND BACKGROUND\nVajont is located in the south-eastern part of the Dolomite Region of the Italian Alps, about 100 km north of Venice (Fig. 1). It was built as a part of the on-going, post-war development of Italy in order to provide HEP for the rapidly-expanding northern cities of Milan, Turin and Modena. Whilst a proposal to site a dam at this location was made in the 1920's, excavation of the sit

In [ ]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)


In [ ]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)


In [ ]:
prediction = pipe.run(
    query="What is the name of dam?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  3.95 Batches/s]


In [ ]:
from pprint import pprint

pprint(prediction)


{'answers': [<Answer {'answer': 'doubly curved arch dam', 'type': 'extractive', 'score': 0.607661247253418, 'context': 'The completed doubly curved arch dam was, at 265.5 metres above the valley floor, the worlds highest thin arch dam. The chord of the dam was 160 m, an', 'offsets_in_document': [{'start': 14, 'end': 36}], 'offsets_in_context': [{'start': 14, 'end': 36}], 'document_ids': ['5c0de1c932ca09dd015ab19a2a775dc9'], 'meta': {'_split_id': 1}}>,
             <Answer {'answer': 'Vajont reservoir', 'type': 'extractive', 'score': 0.34198957681655884, 'context': 'INTRODUCTION\nThe Vajont reservoir disaster is a classic example of the consequences of the failure of engineers and geologists to understand the natur', 'offsets_in_document': [{'start': 17, 'end': 33}], 'offsets_in_context': [{'start': 17, 'end': 33}], 'document_ids': ['32717dd3339bb101abc0d2b3c9a1aaa9'], 'meta': {'_split_id': 0}}>,
             <Answer {'answer': 'The Vaiont Slide', 'type': 'extractive', 'score': 0.039684

In [ ]:
from haystack.utils import print_answers

print_answers(prediction, details="minimum")  ## Choose from `minimum`, `medium`, and `all`


'Query: What is the name of dam?'
'Answers:'
[   {   'answer': 'doubly curved arch dam',
        'context': 'The completed doubly curved arch dam was, at 265.5 metres '
                   'above the valley floor, the worlds highest thin arch dam. '
                   'The chord of the dam was 160 m, an'},
    {   'answer': 'Vajont reservoir',
        'context': 'INTRODUCTION\n'
                   'The Vajont reservoir disaster is a classic example of the '
                   'consequences of the failure of engineers and geologists to '
                   'understand the natur'},
    {   'answer': 'The Vaiont Slide',
        'context': 'ust, 1982, 1992).\n'
                   'REFERENCES\n'
                   'Hendron, A.J., and Patten, F.D, 1985, The Vaiont Slide. US '
                   'Corps of Engineers Technical Report GL-85-8.\n'
                   'Jaegar, C., 1980,'},
    {   'answer': 'storage lake',
        'context': 'he formation upstream of the slide by an uncontrollable '